# Validation Results Recreation (Google Colab Version)

Creates a Validation Result table from Workday and Calabrio data, adapted for Google Colab.

In [ ]:
# --- Google Colab Setup and Configuration ---

# Set this to True if you are running in Google Colab
IS_COLAB_ENV = True # Set to False if running locally

# Base directory for your project in Google Drive or Colab environment
# IMPORTANT: You need to adjust this path based on where you upload your project.
# If using Google Drive, uncomment the lines below and adjust 'your_project_folder':
from google.colab import drive
drive.mount('/content/drive')
# Example: If your 'person-accounts-uploader-colab' folder is directly in MyDrive
PROJECT_BASE_DIR = '/content/drive/MyDrive/person-accounts-uploader-colab'
# If you upload the entire 'person-accounts-uploader-colab' folder to /content/ (less common for persistent storage)
# PROJECT_BASE_DIR = '/content/person-accounts-uploader-colab'

# Relative paths from PROJECT_BASE_DIR
WORKDAY_DATA_RELATIVE_PATH = 'data/workday'
CALABRIO_DATA_RELATIVE_PATH = 'data/calabrio'
CONFIG_RELATIVE_PATH = 'config'
NOTEBOOKS_MODULES_RELATIVE_PATH = 'notebooks_modules'

# Install necessary libraries for Colab
if IS_COLAB_ENV:
    !pip install pandas openpyxl dash jupyter-dash dash-bootstrap-components

import sys
import os
from pathlib import Path
import pandas as pd
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import warnings
from jupyter_dash import JupyterDash # Import JupyterDash for Colab

# Ignore openpyxl UserWarning about default style
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Add module path using the configured PROJECT_BASE_DIR
# This ensures that modules in notebooks_modules can be imported
sys.path.insert(0, str(Path(PROJECT_BASE_DIR, NOTEBOOKS_MODULES_RELATIVE_PATH)))

# Set data file paths using the configured base directory
WORKDAY_DIR = Path(PROJECT_BASE_DIR, WORKDAY_DATA_RELATIVE_PATH)
CALABRIO_DIR = Path(PROJECT_BASE_DIR, CALABRIO_DATA_RELATIVE_PATH)
CONFIG_DIR = Path(PROJECT_BASE_DIR, CONFIG_RELATIVE_PATH)

# Update specific paths used in validation_config
PERSON_ACCOUNTS_DIR = Path(WORKDAY_DIR, 'person_accounts')
PEOPLE_DIR = Path(WORKDAY_DIR, 'people')
USED_ENTRIES_DIR = Path(WORKDAY_DIR, 'used_entries')
ACCOUNT_DATA_PATH = Path(CALABRIO_DIR, 'account_data.json')
PERSON_DATA_PATH = Path(CALABRIO_DIR, 'person_data.json')
CONFIG_DATA_PATH = Path(CALABRIO_DIR, 'config_data.json')

# Import created modules (now that sys.path is updated)
from validation_utils import create_filter_options, safe_get_column
from validation_layout import (
    create_filter_panel, create_validation_grid,
    create_upload_grid, create_app_layout
)
from validation_callbacks import register_callbacks
from validation_api import register_api_callbacks
from validation_calculator import BalanceCalculator

In [ ]:
# Data file paths are now set in the configuration cell above.
# The following lines are commented out as they are redundant.
# person_accounts_path = Path(PERSON_ACCOUNTS_DIR)
# people_path = Path(PEOPLE_DIR)
# used_entries_path = Path(USED_ENTRIES_DIR)
# config_path = Path(CONFIG_DATA_PATH)
# calabrio_path = Path(ACCOUNT_DATA_PATH)
# person_path = Path(PERSON_DATA_PATH)
# balance_rules_path = Path(CONFIG_DIR, 'balance_rules.json')

## 1. Data Loading

### 1.1 Loading Workday Data

In [ ]:
def load_workday_data():
    """Load all Workday data files."""
    workday_df = pd.DataFrame()
    people_df = pd.DataFrame()
    used_entries_df = pd.DataFrame()
    
    # Load person_accounts data
    # person_accounts_path = Path('../data/workday/person_accounts')
    if PERSON_ACCOUNTS_DIR.exists():
        excel_files = list(PERSON_ACCOUNTS_DIR.glob("*.xlsx"))
        if excel_files:
            latest_file = max(excel_files, key=lambda x: x.stat().st_mtime)
            workday_df = pd.read_excel(latest_file, skiprows=6, engine='openpyxl')
            if 'WiserId' in workday_df.columns:
                workday_df['WiserId'] = workday_df['WiserId'].astype(str)
    
    # Load people data
    # people_path = Path('../data/workday/people')
    if PEOPLE_DIR.exists():
        excel_files = list(PEOPLE_DIR.glob("*.xlsx"))
        if excel_files:
            latest_file = max(excel_files, key=lambda x: x.stat().st_mtime)
            people_df = pd.read_excel(latest_file, skiprows=2, engine='openpyxl')
            if 'Latest Headcount Wiser ID' in people_df.columns:
                people_df.rename(columns={'Latest Headcount Wiser ID': 'WiserId'}, inplace=True)
                people_df['WiserId'] = people_df['WiserId'].astype(str)
            if 'Latest Headcount Hire Date' in people_df.columns:
                people_df['Latest Headcount Hire Date'] = pd.to_datetime(people_df['Latest Headcount Hire Date'])
    
    # Load used entries data
    # used_entries_path = Path('../data/workday/used_entries')
    if USED_ENTRIES_DIR.exists():
        excel_files = list(USED_ENTRIES_DIR.glob("*.xlsx"))
        if excel_files:
            latest_file = max(excel_files, key=lambda x: x.stat().st_mtime)
            used_entries_df = pd.read_excel(latest_file, skiprows=6, engine='openpyxl')
            if 'WiserId' in used_entries_df.columns:
                used_entries_df['WiserId'] = used_entries_df['WiserId'].astype(str)
    
    return workday_df, people_df, used_entries_df

workday_df, people_df, used_entries_df = load_workday_data()

### 1.2 Loading Calabrio Data

In [ ]:
import json

def load_calabrio_data():
    """Load all Calabrio data files."""
    calabrio_df = pd.DataFrame()
    person_df = pd.DataFrame()
    
    # Load account data
    # calabrio_path = Path(ACCOUNT_DATA_PATH, "account_data.json")
    if ACCOUNT_DATA_PATH.exists():
        with open(ACCOUNT_DATA_PATH, 'r') as f:
            json_data = json.load(f)
            calabrio_df = pd.DataFrame(json_data)
            if 'EmploymentNumber' in calabrio_df.columns:
                calabrio_df['EmploymentNumber'] = calabrio_df['EmploymentNumber'].astype(str)
            if 'Accrued' in calabrio_df.columns:
                calabrio_df['Accrued'] = pd.to_numeric(calabrio_df['Accrued'], errors='coerce').fillna(0)
    
    # Load person data
    # person_path = Path(PERSON_DATA_PATH, "person_data.json")
    if PERSON_DATA_PATH.exists():
        with open(PERSON_DATA_PATH, 'r') as f:
            json_data = json.load(f)
            person_df = pd.DataFrame(json_data)
            if 'EmploymentNumber' in person_df.columns:
                person_df['EmploymentNumber'] = person_df['EmploymentNumber'].astype(str)
            person_df['EmploymentStartDate'] = pd.to_datetime(person_df['EmploymentStartDate'])
    
    return calabrio_df, person_df

calabrio_df, calabrio_person_df = load_calabrio_data()

## 2. Data Preprocessing

### 2.1 Workday Data Preprocessing

In [ ]:
# Merge Workday data with people data
if not workday_df.empty and not people_df.empty:
    if ('WiserId' in workday_df.columns and 'WiserId' in people_df.columns and
        'Latest Headcount Hire Date' in people_df.columns and 
        'Latest Headcount Primary Work Email' in people_df.columns):
        workday_df = pd.merge(
            workday_df,
            people_df[['WiserId', 'Latest Headcount Hire Date', 'Latest Headcount Primary Work Email']],
            on='WiserId',
            how='left'
        )

# Add MappedEmploymentNumber (using WiserId as fallback)
workday_df["MappedEmploymentNumber"] = workday_df["WiserId"]


### 2.2 Calabrio Data Preprocessing

In [ ]:
# Merge person data with calabrio_df
if not calabrio_df.empty and not calabrio_person_df.empty:
    if 'EmploymentNumber' in calabrio_df.columns and 'EmploymentNumber' in calabrio_person_df.columns:
        calabrio_df = pd.merge(
            calabrio_df,
            calabrio_person_df[['EmploymentNumber', 'PersonId', 'BusinessUnitName', 'EmploymentStartDate']],
            on='EmploymentNumber',
            how='left',
            suffixes=('', '_person')
        )

# Load config data for absence mapping
# config_path = Path('data/calabrio/config_data.json')
if CONFIG_DATA_PATH.exists():
    with open(CONFIG_DATA_PATH, 'r') as f:
        config_data = json.load(f)


## 3. Creating the Validation Table

In [ ]:
# Function to map AbsenceId
def map_absence_id(row, config_data):
    """
    Retrieves AbsenceId from config_data using BusinessUnitName and AbsenceName.
    
    Args:
        row: DataFrame row
        config_data: Configuration data dictionary
    
    Returns:
        str: AbsenceId, or None if not found
    """
    business_unit = row['BusinessUnitName']
    absence_name = row['AbsenceName']
    
    # Check if BusinessUnitName exists in config data
    if business_unit not in config_data:
        # If not found, standardize using mapping function
        from notebooks_modules.validation_utils import map_business_unit
        mapped_bu = map_business_unit(business_unit)
        if mapped_bu is None or mapped_bu not in config_data:
            return None
        business_unit = mapped_bu
    
    # Get absences list
    absences = config_data.get(business_unit, {}).get('absences', {}).get('Result', [])
    
    # Attempt exact match
    for absence in absences:
        if absence.get('Name') == absence_name:
            return absence.get('Id')
    
    # Attempt case-insensitive match
    absence_name_lower = absence_name.lower()
    for absence in absences:
        if absence.get('Name', '').lower() == absence_name_lower:
            return absence.get('Id')
    
    # Attempt partial match
    for absence in absences:
        if absence_name_lower in absence.get('Name', '').lower():
            return absence.get('Id')
    
    # Retry by adding 'Global -' prefix
    if not absence_name.lower().startswith('global -'):
        global_absence = f"Global - {absence_name}"
        for absence in absences:
            if absence.get('Name') == global_absence:
                return absence.get('Id')
    
    return None

# Add AbsenceId column to calabrio_df
def add_absence_id_to_df(df, config_data):
    """
    Adds an AbsenceId column to the DataFrame.
    
    Args:
        df: Input DataFrame
        config_data: Configuration data dictionary
    
    Returns:
        DataFrame: DataFrame with AbsenceId column added
    """
    # List to store results
    absence_ids = []
    
    # Process each row
    for _, row in df.iterrows():
        absence_id = map_absence_id(row, config_data)
        absence_ids.append(absence_id)
    
    # Create result DataFrame
    result_df = df.copy()
    result_df['AbsenceId'] = absence_ids
    
    # Mapping results statistics
    mapped_count = sum(1 for aid in absence_ids if aid is not None)
    total_count = len(absence_ids)
    print(f"Mapping results: {mapped_count}/{total_count} ({mapped_count/total_count*100:.1f}%) records mapped with AbsenceId")

    

    return result_df



# Add AbsenceId

calabrio_df_with_id = add_absence_id_to_df(calabrio_df, config_data)



# Verify results

print(f"DataFrame with AbsenceId added:")

print(calabrio_df_with_id[['BusinessUnitName', 'AbsenceName', 'AbsenceId']].head())



# Check examples of unmapped AbsenceIds

missing_ids = calabrio_df_with_id[calabrio_df_with_id['AbsenceId'].isna()]

if not missing_ids.empty:

    print(f"\nExamples of unmapped AbsenceIds ({len(missing_ids)} items):")

    print(missing_ids[['BusinessUnitName', 'AbsenceName']].head(10))

    

    # Aggregate combinations of BusinessUnitName and AbsenceName

    missing_combinations = missing_ids.groupby(['BusinessUnitName', 'AbsenceName']).size().reset_index(name='count')

    missing_combinations = missing_combinations.sort_values('count', ascending=False)

    print("\nMost frequent unmapped combinations:")

    print(missing_combinations.head(10))
def create_validation_table(workday_df, calabrio_df):

    """Create validation table comparing Workday and Calabrio data."""

    # Clean and convert columns

    workday_df = workday_df.copy()

    calabrio_df = calabrio_df.copy()

    

    workday_df["WiserId"] = workday_df["WiserId"].fillna("").astype(str).str.strip().str.lower()

    workday_df["Original_AbsenceType_Case"] = workday_df["AbsenceType"].fillna("").astype(str).str.strip()

    workday_df["AbsenceType"] = workday_df["AbsenceType"].fillna("").astype(str).str.strip().str.lower()

    

    calabrio_df["EmploymentNumber"] = calabrio_df["EmploymentNumber"].fillna("").astype(str).str.strip().str.lower()

    calabrio_df["AbsenceName"] = calabrio_df["AbsenceName"].fillna("").astype(str).str.strip().str.lower()

    

    # Group Calabrio data

    calabrio_grouped = calabrio_df.groupby(["EmploymentNumber", "AbsenceName"]).first().reset_index()

    

    # Merge data

    merged_df = pd.merge(

        workday_df,

        calabrio_grouped,

        left_on=["MappedEmploymentNumber", "AbsenceType"],

        right_on=["EmploymentNumber", "AbsenceName"],

        how="left"

    )

    

    # Create display DataFrame

    display_df = pd.DataFrame({

        "Workday Person Number": merged_df["WiserId"].fillna(merged_df["EmploymentNumber"]),

        "Calabrio Person Number": merged_df["EmploymentNumber"],

        "Workday Absence Type": merged_df["Original_AbsenceType_Case"],

        "Calabrio Absence Type": merged_df["AbsenceName"],

        "Absence ID": merged_df["AbsenceId"],

        "Calabrio BusinessUnitName": merged_df["BusinessUnitName"],

        "StartDate": merged_df["StartDate"],

        "ContractName": merged_df["ContractName"],

        "Calabrio Balance In": pd.to_numeric(merged_df["BalanceIn"], errors="coerce").fillna(0).round().astype("Int64"),

        "Calabrio_Accrued": pd.to_numeric(merged_df["Accrued"].fillna(0), errors="coerce").fillna(0).round().astype("Int64"),

        "Calabrio Extra": pd.to_numeric(merged_df["Extra"].fillna(0), errors="coerce").fillna(0).round().astype("Int64"),

        "Units Approved": pd.to_numeric(safe_get_column(merged_df, ["Units Approved"], 0), errors="coerce").fillna(0).round().astype("Int64"),

        "TrackedBy": merged_df["TrackedBy"],

        "Calabrio PersonId": merged_df["PersonId"],

        "Beginning Year Balance": pd.to_numeric(safe_get_column(merged_df, ["Beginning Year Balance"], 0), errors="coerce").fillna(0),

        "Accrued this year": pd.to_numeric(safe_get_column(merged_df, ["Accrued this year"], 0), errors="coerce").fillna(0),

        "Latest Headcount Primary Work Email": merged_df["Latest Headcount Primary Work Email"],

        "EmploymentStartDate": merged_df["EmploymentStartDate"],

        "Latest Headcount Hire Date": merged_df["Latest Headcount Hire Date"]

    })

    

    # Calculate correct values

    # balance_rules_path is now defined in the configuration cell

    calculator = BalanceCalculator(Path(CONFIG_DIR, 'balance_rules.json'))

    display_df["Correct Balance In"], display_df["Correct_Accrued"] = zip(

        *display_df.apply(lambda row: calculator.calculate_correct_values(row, row["Workday Absence Type"]), axis=1)

    )

    

    # Calculate matches

    display_df["Balance Match"] = display_df.apply(

        lambda row: "✅" if row["Correct Balance In"] == row["Calabrio Balance In"] else "❌", axis=1

    )

    display_df["Accrual Match"] = display_df.apply(

        lambda row: "✅" if row["Correct_Accrued"] == row["Calabrio_Accrued"] else "❌", axis=1

    )

    

    # Calculate balance difference

    display_df["Balance Difference"] = display_df["Correct Balance In"] - display_df["Calabrio Balance In"]

    

    return display_df



validation_df = create_validation_table(workday_df, calabrio_df_with_id)
## 4. Creating the Dash Application
# Initialize Dash application using JupyterDash for Colab compatibility

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])



# Create filter options

filter_options = create_filter_options(validation_df)



# Create components

filter_panel = create_filter_panel(filter_options)

validation_grid = create_validation_grid(validation_df)

upload_grid = create_upload_grid()



# Set layout

app.layout = create_app_layout(filter_panel, validation_grid, upload_grid)



# Register callbacks

register_callbacks(app, validation_df)

register_api_callbacks(app)
# Activate
# Run the Dash application in inline mode for Google Colab

app.run_server(mode='inline', debug=True, port=8050)


Mapping results: 22323/22323 (100.0%) records mapped with AbsenceId
DataFrame with AbsenceId added:
  BusinessUnitName                          AbsenceName  \
0               CS   EST - Study Leave with Average Pay   
1               CS             EST - Unpaid Study Leave   
2               CS  Global - Compassionate Leave (Days)   
3               CS                      Global - Me Day   
4               CS               Global - Volunteer Day   

                              AbsenceId  
0  253e965b-1182-45a8-adaf-b03100eccc22  
1  f783102b-0c9d-425e-97c8-b03100eccc22  
2  008e44c8-d1cc-48c1-810f-b03100eccc22  
3  ef2b69b6-b9e5-44cc-afe9-b03400de9a6f  
4  674eb417-2204-41cc-9106-b03100eccc22  
